In [1]:
# !pip install Praw
# !pip install --upgrade praw

In [2]:
import sys
!{sys.executable} -m pip install praw
!{sys.executable} -m pip install psaw
import praw
import pandas as pd
import datetime as dt
import json

f = open("../secret_python.txt", "r")
client_id = f.readline().rstrip()
secret_key = f.readline().rstrip()
f.close()

## Praw

In [3]:
reddit = praw.Reddit(client_id=client_id, \
                     client_secret=secret_key, \
                     user_agent='Scrape')

reddit.read_only

True

In [4]:
dank_meme = reddit.subreddit('dank_meme')

In [5]:
memesDict = {}
memesDict["_default"] = {}

Source: https://stackoverflow.com/questions/40960449/getting-all-submissions-for-the-past-two-months-from-a-particular-subreddit-usi

In [6]:
iterator = 1
for submission in dank_meme.hot(limit = 100):
    #print(dt.datetime.fromtimestamp(submission.created_utc)) 
    memesDict["_default"][iterator] = {"title":submission.title, "thumbnail":{"thumbnail":submission.thumbnail, "height":submission.thumbnail_height, "width":submission.thumbnail_width}, "created_utc":submission.created_utc, "author":submission.author.name, "id":submission.id, "ups":submission.ups, "downs":submission.downs, "media": submission.url}
    iterator +=1

In [7]:
print(memesDict["_default"][2])

{'title': 'This is an avengers level relief', 'thumbnail': {'thumbnail': 'https://b.thumbs.redditmedia.com/pOVMmHaB4d0UvZM0jJr372ulpY70lhz3WdqhaO8vteA.jpg', 'height': 140, 'width': 140}, 'created_utc': 1586616386.0, 'author': 'dankbob_memepants_', 'id': 'fz6p05', 'ups': 9887, 'downs': 0, 'media': 'https://i.redd.it/repxzfelb7s41.jpg'}


In [8]:
with open('memesDatabase.json', 'w') as json_file:
    json.dump(memesDict, json_file)

In [9]:
#Used to find out what fields does an object have
import pprint
#pprint.pprint(vars(submission))

## Pushshift

With help from the following link: https://medium.com/@RareLoot/using-pushshifts-api-to-extract-reddit-submissions-fb517b286563

In [10]:
import requests

In [11]:
base_url = "https://api.pushshift.io/"
fields="author,subreddit,id,created_utc,all_awardings,score,thumbnail,thumbnail_height,thumbnail_width,title,url,full_link"
def queryBySub(sub, before="30d", after="60d", sort_type="score", sort="asc",size="25", fields=fields):
    url = base_url + "reddit/search/submission/?subreddit="+sub+"&sort="+sort+"&sort_type="+sort_type+"&size="+size+"&after="+after+"&before="+before+"&fields="+fields
    response = requests.get(url)
    return json.loads(response.text)
data = queryBySub("dank_meme")

In [12]:
data["data"]

[{'all_awardings': [],
  'author': 'Popcorny_YT',
  'created_utc': 1583363896,
  'full_link': 'https://www.reddit.com/r/dank_meme/comments/fdlrju/hmmmm_yes_gold_robbery/',
  'id': 'fdlrju',
  'score': 0,
  'subreddit': 'dank_meme',
  'thumbnail': 'https://b.thumbs.redditmedia.com/5OkmOvdbo7IC86JCv8gkwH6EkBIuvgMnypM0LCua_2c.jpg',
  'thumbnail_height': 140,
  'thumbnail_width': 140,
  'title': 'Hmmmm yes gold robbery',
  'url': 'https://i.redd.it/5vldqep9oqk41.jpg'},
 {'all_awardings': [],
  'author': 'hex272727',
  'created_utc': 1583960019,
  'full_link': 'https://www.reddit.com/r/dank_meme/comments/fh46fl/how_to_manipulate_your_wife_to_get_bigger_boobs/',
  'id': 'fh46fl',
  'score': 0,
  'subreddit': 'dank_meme',
  'thumbnail': 'https://b.thumbs.redditmedia.com/SZ6rRxmzL6hq4GtqBTslU4bWb5nliGbOZpiciS9xYqY.jpg',
  'thumbnail_height': 105,
  'thumbnail_width': 140,
  'title': 'how to manipulate your wife to get bigger boobs',
  'url': 'https://youtu.be/Id1yPS7wvwk'},
 {'all_awardings': 

## Python Pushshift.io API Wrapper 

API: https://github.com/dmarx/psaw

In [13]:
import psaw
from psaw import PushshiftAPI

api = PushshiftAPI(reddit)

In [14]:
current_time = dt.datetime.now()
end_date = current_time-dt.timedelta(days = 30) #how many days before today you want the search to end
start_date = current_time-dt.timedelta(days = 33) #how many days before today do you want the search to start
start_epoch = int(dt.datetime.timestamp(start_date))
end_epoch = int(dt.datetime.timestamp(end_date))

In [15]:
subr = 'dank_meme'
searchResults = api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit=subr)
cache = []

for result in searchResults:
    cache.append(result)
    
cache[0:10]

[Submission(id='fi48b6'),
 Submission(id='fi46bw'),
 Submission(id='fi45v3'),
 Submission(id='fi441s'),
 Submission(id='fi43zf'),
 Submission(id='fi43eh'),
 Submission(id='fi42ry'),
 Submission(id='fi42en'),
 Submission(id='fi428y'),
 Submission(id='fi3wpd')]

In [16]:
#pprint.pprint(vars(cache[1]))

In [17]:
memesDict2 = {}
memesDict2["_default"] = {}

In [18]:
iterator = 1
maxUpvote = 0
for submission in cache:
    if (submission is not None and submission.is_video is False): #ignore video memes
        #print(dt.datetime.fromtimestamp(submission.created_utc)) 
        if (submission.ups > maxUpvote):
            maxUpvote = submission.ups
        memesDict2["_default"][iterator] = {"title":submission.title, "thumbnail":{"thumbnail":submission.thumbnail, "height":submission.thumbnail_height, "width":submission.thumbnail_width}, "created_utc":submission.created_utc, "id":submission.id, "ups":submission.ups, "downs":submission.downs, "media": submission.url, "url": submission.permalink, "awards": submission.all_awardings, 'subreddit':submission.subreddit.display_name}
        if (submission.author is not None): #in case the account is deleted
            memesDict2["_default"][iterator]["author"] = submission.author.name 
        else:
            memesDict2["_default"][iterator]["author"] = "N/A"
        iterator +=1

In [19]:
memesDict2['_default'][2]

{'title': 'Do you are have stuipid',
 'thumbnail': {'thumbnail': 'https://b.thumbs.redditmedia.com/F3YZ5Q8KagWE39-UoQy8rjh53hYIZGQn3qNZIe87cyE.jpg',
  'height': 140,
  'width': 140},
 'created_utc': 1584123697.0,
 'id': 'fi46bw',
 'ups': 6,
 'downs': 0,
 'media': 'https://i.redd.it/e9qawdtjfhm41.png',
 'url': '/r/dank_meme/comments/fi46bw/do_you_are_have_stuipid/',
 'awards': [],
 'subreddit': 'dank_meme',
 'author': 'bilozaooka'}

There are some thumbnails with value as 'default' btw.

In [20]:
subr = 'meme'
searchResults = api.search_submissions(after=start_epoch,
                            before=end_epoch,
                            subreddit=subr)
cache = []

for result in searchResults:
    cache.append(result)
    
cache[0:10]

[Submission(id='fi4dxt'),
 Submission(id='fi4dsc'),
 Submission(id='fi4dlw'),
 Submission(id='fi4djr'),
 Submission(id='fi4dgw'),
 Submission(id='fi4d6d'),
 Submission(id='fi4d0j'),
 Submission(id='fi4cwk'),
 Submission(id='fi4crr'),
 Submission(id='fi4cc0')]

In [21]:
for submission in cache:
    if (submission is not None and submission.is_video is False):
        #print(dt.datetime.fromtimestamp(submission.created_utc)) 
        if (submission.ups > maxUpvote):
            maxUpvote = submission.ups
        memesDict2["_default"][iterator] = {"title":submission.title, "thumbnail":{"thumbnail":submission.thumbnail, "height":submission.thumbnail_height, "width":submission.thumbnail_width}, "created_utc":submission.created_utc, "id":submission.id, "ups":submission.ups, "downs":submission.downs, "media": submission.url, "url": submission.permalink, "awards": submission.all_awardings}
        if (submission.author is not None):
            memesDict2["_default"][iterator]["author"] = submission.author.name 
        else:
            memesDict2["_default"][iterator]["author"] = "N/A"
        iterator +=1

In [22]:
maxUpvote

33036

In [23]:
len(memesDict2['_default'])

9381

In [24]:
with open('memesDatabase2.json', 'w') as json_file:
    json.dump(memesDict2, json_file)